In [1]:
!pip install deep-translator

    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [3]:
from deep_translator import GoogleTranslator


def translate_text(text, target_language):

    if target_language not in ['en', 'ar']:
        raise ValueError(
            "Target language must be 'en' for English or 'ar' for Arabic.")

    translated = GoogleTranslator(
        source='auto', 
        target=target_language).translate(text)
    return translated


# Example usage:
text_en = "Hello, how are you?"
text_ar = "مرحبًا، كيف حالك؟"

translated_to_ar = translate_text(text_en, 'ar')
translated_to_en = translate_text(text_ar, 'en')

print(f"Translated to Arabic: {translated_to_ar}")
print(f"Translated to English: {translated_to_en}")

Translated to Arabic: مرحبا، كيف حالك؟
Translated to English: Hello, how are you?


In [15]:
import googlemaps
import time

# Replace with your actual Google Maps API key
gmaps = googlemaps.Client(key=API_KEY)


def fetch_places_for_location(latitude, longitude, radius, keyword):

    location = (latitude, longitude)

    places_result = gmaps.places_nearby(

        location=location, radius=radius, keyword=keyword)

    branches = []


    while True:

        for place in places_result['results']:

            branch_name = place['name']

            branch_location = place['geometry']['location']

            branch_address = place.get('vicinity', 'Address not available')


            maps_url = f"https://www.google.com/maps/search/?api=1&query={branch_location['lat']},{branch_location['lng']}"


            branches.append({

                "name": branch_name,

                "location": (branch_location['lat'], branch_location['lng']),

                "address": branch_address,

                "maps_url": maps_url

            })


        next_page_token = places_result.get('next_page_token')

        if not next_page_token:

            break


        time.sleep(2)

        places_result = gmaps.places_nearby(

            location=location, radius=radius, keyword=keyword, page_token=next_page_token)

    return branches



def find_nearby_branches(latitude, longitude, radius=5000, keyword="Banque Misr", grid_size=4):

    branches = []

    # Approx. conversion of meters to degrees (latitude)

    lat_step = radius / 111320

    # Approx. conversion of meters to degrees (longitude)

    lng_step = radius / (40075000 * abs(latitude) / 360)


    # Generate grid points based on the central location

    for i in range(-grid_size, grid_size + 1):

        for j in range(-grid_size, grid_size + 1):

            lat = latitude + i * lat_step

            lng = longitude + j * lng_step

            location_branches = fetch_places_for_location(

                lat, lng, radius, keyword)

            branches.extend(location_branches)

    return branches



# Example usage: Assuming you have the user's latitude and longitude

user_latitude = 30.0444  # Example latitude (Cairo, Egypt)

user_longitude = 31.2357  # Example longitude (Cairo, Egypt)


# Find all nearby Bank Misr branches within a larger area

all_branches = find_nearby_branches(user_latitude, user_longitude)


# Remove duplicates

unique_branches_4 = {

    f"{branch['location'][0]},{branch['location'][1]}": branch for branch in all_branches}


# Print the details of all branches with Google Maps links

for branch in unique_branches_4.values():

    print(f"Branch Name: {branch['name']}")

    print(f"Location: {branch['location']}")

    print(f"Address: {branch['address']}")

    print(f"Google Maps Link: {branch['maps_url']}\n")

Branch Name: Bank misr atm
Location: (29.8980716, 31.2770744)
Address: V7XG+6RG, Madinet Al Hawamdeyah, El Hawamdeya
Google Maps Link: https://www.google.com/maps/search/?api=1&query=29.8980716,31.2770744

Branch Name: بنك مصر فرع البدرشين
Location: (29.8463797, 31.2770966)
Address: R7WG+GRV, Unnamed Rd, Madinet Al Badrashin, Badrshein
Google Maps Link: https://www.google.com/maps/search/?api=1&query=29.8463797,31.2770966

Branch Name: Banque Misr
Location: (29.9088558, 31.2894392)
Address: 10 ش كورنيش النيل، الكورنيش، اسوان
Google Maps Link: https://www.google.com/maps/search/?api=1&query=29.9088558,31.2894392

Branch Name: Banque Misr بنك مصر فرع سرايات المعادي
Location: (29.9567564, 31.26097619999999)
Address: 69 Rd 9, Maadi Al Khabiri Ash Sharqeyah, Maadi
Google Maps Link: https://www.google.com/maps/search/?api=1&query=29.9567564,31.26097619999999

Branch Name: بنك مصر
Location: (29.9553117, 31.2762433)
Address: شارع ١٩٩ - قسم شرطة المعادي، حي ثكنات, Maadi
Google Maps Link: https:

In [16]:
len(unique_branches_4)

108

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame(list(unique_branches_4.values()))

In [19]:
df.head()

,name,location,address,maps_url
0,Bank misr atm,"(29.8980716, 31.2770744)","V7XG+6RG, Madinet Al Hawamdeyah, El Hawamdeya",https://www.google.com/maps/search/?api=1&quer...
1,بنك مصر فرع البدرشين,"(29.8463797, 31.2770966)","R7WG+GRV, Unnamed Rd, Madinet Al Badrashin, Ba...",https://www.google.com/maps/search/?api=1&quer...
2,Banque Misr,"(29.9088558, 31.2894392)",10 ش كورنيش النيل، الكورنيش، اسوان,https://www.google.com/maps/search/?api=1&quer...
3,Banque Misr بنك مصر فرع سرايات المعادي,"(29.9567564, 31.26097619999999)","69 Rd 9, Maadi Al Khabiri Ash Sharqeyah, Maadi",https://www.google.com/maps/search/?api=1&quer...
4,بنك مصر,"(29.9553117, 31.2762433)","شارع ١٩٩ - قسم شرطة المعادي، حي ثكنات, Maadi",https://www.google.com/maps/search/?api=1&quer...


In [22]:
df.to_csv('branches_locations.csv', index=False)